In [0]:
# semi-structure json file to -> structure data file

dbutils.fs.put("/schenarios/complex_json.json","""[{
	"id": "0001",
	"type": "donut",
	"name": "Cake",
	"ppu": 0.55,
	"batters":
		{
			"batter":
				[
					{ "id": "1001", "type": "Regular" },
					{ "id": "1002", "type": "Chocolate" },
					{ "id": "1003", "type": "Blueberry" },
					{ "id": "1004", "type": "Devil's Food" }
				]
		},
	"topping":
		[
			{ "id": "5001", "type": "None" },
			{ "id": "5002", "type": "Glazed" },
			{ "id": "5005", "type": "Sugar" },
			{ "id": "5007", "type": "Powdered Sugar" },
			{ "id": "5006", "type": "Chocolate with Sprinkles" },
			{ "id": "5003", "type": "Chocolate" },
			{ "id": "5004", "type": "Maple" }
		]
}]""",True)

Wrote 608 bytes.


True

In [0]:
json_df = spark.read.option("multiline", "true").json("/schenarios/complex_json.json")
json_df.printSchema()
json_df.display()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



batters,id,name,ppu,topping,type
"List(List(List(1001, Regular), List(1002, Chocolate), List(1003, Blueberry), List(1004, Devil's Food)))",0001,Cake,0.55,"List(List(5001, None), List(5002, Glazed), List(5005, Sugar), List(5007, Powdered Sugar), List(5006, Chocolate with Sprinkles), List(5003, Chocolate), List(5004, Maple))",donut


In [0]:
from pyspark.sql.functions import col, explode
json_df_final = json_df.withColumn("batter_explode", explode("batters.batter"))\
    .withColumn("batter_id", col("batter_explode.id"))\
    .withColumn("batter_type", col("batter_explode.type"))\
    .drop("batters", "batter_explode")
json_df_final.display()

id,name,ppu,topping,type,batter_id,batter_type
0001,Cake,0.55,"List(List(5001, None), List(5002, Glazed), List(5005, Sugar), List(5007, Powdered Sugar), List(5006, Chocolate with Sprinkles), List(5003, Chocolate), List(5004, Maple))",donut,1001,Regular
0001,Cake,0.55,"List(List(5001, None), List(5002, Glazed), List(5005, Sugar), List(5007, Powdered Sugar), List(5006, Chocolate with Sprinkles), List(5003, Chocolate), List(5004, Maple))",donut,1002,Chocolate
0001,Cake,0.55,"List(List(5001, None), List(5002, Glazed), List(5005, Sugar), List(5007, Powdered Sugar), List(5006, Chocolate with Sprinkles), List(5003, Chocolate), List(5004, Maple))",donut,1003,Blueberry
0001,Cake,0.55,"List(List(5001, None), List(5002, Glazed), List(5005, Sugar), List(5007, Powdered Sugar), List(5006, Chocolate with Sprinkles), List(5003, Chocolate), List(5004, Maple))",donut,1004,Devil's Food


In [0]:
json_df_final = json_df_final.withColumn("topping_explode", explode("topping"))\
    .withColumn("topping_id", col("topping_explode.id"))\
    .withColumn("topping_type", col("topping_explode.type"))\
    .drop("topping_explode", "topping")

json_df_final.limit(10).display()

id,name,ppu,type,batter_id,batter_type,topping_id,topping_type
0001,Cake,0.55,donut,1001,Regular,5001,None
0001,Cake,0.55,donut,1001,Regular,5002,Glazed
0001,Cake,0.55,donut,1001,Regular,5005,Sugar
0001,Cake,0.55,donut,1001,Regular,5007,Powdered Sugar
0001,Cake,0.55,donut,1001,Regular,5006,Chocolate with Sprinkles
0001,Cake,0.55,donut,1001,Regular,5003,Chocolate
0001,Cake,0.55,donut,1001,Regular,5004,Maple
0001,Cake,0.55,donut,1001,Regular,5001,None
0001,Cake,0.55,donut,1001,Regular,5002,Glazed
0001,Cake,0.55,donut,1001,Regular,5005,Sugar
